#Queistion 1- Part d


In [ ]:
import numpy as np
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot
from keras import backend
from keras.models import load_model
from keras.datasets.mnist import load_data
import matplotlib.pyplot as plt
import tensorflow

>Discriminatir & Custom Activation 

In [ ]:
def custom_activation(output):
	logexpsum = backend.sum(backend.exp(output), axis=-1, keepdims=True)
	result = logexpsum / (logexpsum + 1.0)
	return result

def define_discriminator(in_shape=(28,28,1), n_classes=10):
	in_image = layers.Input(shape=in_shape)
	disc = layers.Conv2D(128, (3,3), strides=(2,2), padding='same')(in_image)
	disc = layers.LeakyReLU(alpha=0.2)(disc)
	disc = layers.Conv2D(128, (3,3), strides=(2,2), padding='same')(disc)
	disc = layers.LeakyReLU(alpha=0.2)(disc)
	disc = layers.Conv2D(128, (3,3), strides=(2,2), padding='same')(disc)
	disc = layers.LeakyReLU(alpha=0.2)(disc)
	disc = layers.Flatten()(disc)
	disc = layers.Dropout(0.4)(disc)
	disc = layers.Dense(n_classes)(disc)
	c_out_layer = layers.Activation('softmax')(disc)
	c_model = models.Model(in_image, c_out_layer)
	c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
	d_out_layer = layers.Lambda(custom_activation)(disc)
	d_model = models.Model(in_image, d_out_layer)
	d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
	return d_model, c_model

> VAE
>>encoder part

In [ ]:
img_size = 28
num_channels = 1
latent_space_dim = 2


x = layers.Input(shape=(img_size, img_size, num_channels), name="encoder_input")

encoder_conv_layer1 = layers.Conv2D(filters=1, kernel_size=(3, 3), padding="same", strides=1, name="encoder_conv_1")(x)
encoder_norm_layer1 = layers.BatchNormalization(name="encoder_norm_1")(encoder_conv_layer1)
encoder_activ_layer1 = layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_norm_layer1)

encoder_conv_layer2 = layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", strides=1, name="encoder_conv_2")(encoder_activ_layer1)
encoder_norm_layer2 = layers.BatchNormalization(name="encoder_norm_2")(encoder_conv_layer2)
encoder_activ_layer2 = layers.LeakyReLU(name="encoder_activ_layer_2")(encoder_norm_layer2)

encoder_conv_layer3 = layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_3")(encoder_activ_layer2)
encoder_norm_layer3 = layers.BatchNormalization(name="encoder_norm_3")(encoder_conv_layer3)
encoder_activ_layer3 = layers.LeakyReLU(name="encoder_activ_layer_3")(encoder_norm_layer3)

encoder_conv_layer4 = layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_4")(encoder_activ_layer3)
encoder_norm_layer4 = layers.BatchNormalization(name="encoder_norm_4")(encoder_conv_layer4)
encoder_activ_layer4 = layers.LeakyReLU(name="encoder_activ_layer_4")(encoder_norm_layer4)

encoder_conv_layer5 = layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=1, name="encoder_conv_5")(encoder_activ_layer4)
encoder_norm_layer5 = layers.BatchNormalization(name="encoder_norm_5")(encoder_conv_layer5)
encoder_activ_layer5 = layers.LeakyReLU(name="encoder_activ_layer_5")(encoder_norm_layer5)

shape_before_flatten = tensorflow.keras.backend.int_shape(encoder_activ_layer5)[1:]
encoder_flatten = layers.Flatten()(encoder_activ_layer5)

encoder_mu = layers.Dense(units=latent_space_dim, name="encoder_mu")(encoder_flatten)
encoder_log_variance = layers.Dense(units=latent_space_dim, name="encoder_log_variance")(encoder_flatten)

encoder_mu_log_variance_model = models.Model(x, (encoder_mu, encoder_log_variance), name="encoder_mu_log_variance_model")
def sampling(mu_log_variance):
    mu, log_variance = mu_log_variance
    epsilon = tensorflow.keras.backend.random_normal(shape=tensorflow.keras.backend.shape(mu), mean=0.0, stddev=1.0)
    random_sample = mu + tensorflow.keras.backend.exp(log_variance/2) * epsilon
    return random_sample

encoder_output = layers.Lambda(sampling, name="encoder_output")([encoder_mu, encoder_log_variance])

encoder = models.Model(x, encoder_output, name="encoder_model")
encoder.summary()

Model: "encoder_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 encoder_conv_1 (Conv2D)        (None, 28, 28, 1)    10          ['encoder_input[0][0]']          
                                                                                                  
 encoder_norm_1 (BatchNormaliza  (None, 28, 28, 1)   4           ['encoder_conv_1[0][0]']         
 tion)                                                                                            
                                                                                                  
 encoder_leakyrelu_1 (LeakyReLU  (None, 28, 28, 1)   0           ['encoder_norm_1[0][0

>VAE
>>decoder part

In [ ]:
decoder_input = layers.Input(shape=(latent_space_dim), name="decoder_input")
decoder_dense_layer1 = layers.Dense(units=np.prod(shape_before_flatten), name="decoder_dense_1")(decoder_input)
decoder_reshape = layers.Reshape(target_shape=shape_before_flatten)(decoder_dense_layer1)

decoder_conv_tran_layer1 = layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), padding="same", strides=1, name="decoder_conv_tran_1")(decoder_reshape)
decoder_norm_layer1 = layers.BatchNormalization(name="decoder_norm_1")(decoder_conv_tran_layer1)
decoder_activ_layer1 = layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_norm_layer1)

decoder_conv_tran_layer2 = layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), padding="same", strides=2, name="decoder_conv_tran_2")(decoder_activ_layer1)
decoder_norm_layer2 = layers.BatchNormalization(name="decoder_norm_2")(decoder_conv_tran_layer2)
decoder_activ_layer2 = layers.LeakyReLU(name="decoder_leakyrelu_2")(decoder_norm_layer2)

decoder_conv_tran_layer3 = layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), padding="same", strides=2, name="decoder_conv_tran_3")(decoder_activ_layer2)
decoder_norm_layer3 = layers.BatchNormalization(name="decoder_norm_3")(decoder_conv_tran_layer3)
decoder_activ_layer3 = layers.LeakyReLU(name="decoder_leakyrelu_3")(decoder_norm_layer3)

decoder_conv_tran_layer4 = layers.Conv2DTranspose(filters=1, kernel_size=(3, 3), padding="same", strides=1, name="decoder_conv_tran_4")(decoder_activ_layer3)
decoder_output = layers.LeakyReLU(name="decoder_output")(decoder_conv_tran_layer4 )

decoder = models.Model(decoder_input, decoder_output, name="decoder_model")
decoder.summary()

Model: "decoder_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 decoder_dense_1 (Dense)     (None, 3136)              9408      
                                                                 
 reshape (Reshape)           (None, 7, 7, 64)          0         
                                                                 
 decoder_conv_tran_1 (Conv2D  (None, 7, 7, 64)         36928     
 Transpose)                                                      
                                                                 
 decoder_norm_1 (BatchNormal  (None, 7, 7, 64)         256       
 ization)                                                        
                                                                 
 decoder_leakyrelu_1 (LeakyR  (None, 7, 7, 64)       

>VAE
>>total vae (encoder part + decoder part)

In [ ]:
vae_input = layers.Input(shape=(img_size, img_size, num_channels), name="VAE_input")
vae_encoder_output = encoder(vae_input)
vae_decoder_output = decoder(vae_encoder_output)
vae = models.Model(vae_input, vae_decoder_output, name="VAE")
vae.summary()

Model: "VAE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 VAE_input (InputLayer)      [(None, 28, 28, 1)]       0         
                                                                 
 encoder_model (Functional)  (None, 2)                 106130    
                                                                 
 decoder_model (Functional)  (None, 28, 28, 1)         121537    
                                                                 
Total params: 227,667
Trainable params: 226,833
Non-trainable params: 834
_________________________________________________________________


In [ ]:
vae1_input = layers.Input(shape=(img_size, img_size, num_channels), name="vae1_input")
vae1_encoder_output = encoder(vae_input)
vae1_decoder_output = decoder(vae_encoder_output)
vae1 = models.Model(vae_input, vae_decoder_output, name="vae1")
vae1.summary()

Model: "VAE2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 VAE_input (InputLayer)      [(None, 28, 28, 1)]       0         
                                                                 
 encoder_model (Functional)  (None, 2)                 106130    
                                                                 
 decoder_model (Functional)  (None, 28, 28, 1)         121537    
                                                                 
Total params: 227,667
Trainable params: 226,833
Non-trainable params: 834
_________________________________________________________________


>loss


In [ ]:
def loss_function(encoder_mu, encoder_log_variance):
    def vae_reconstruction_loss(X_train, y_predict):
        reconstruction_loss_factor = 1000
        reconstruction_loss = tensorflow.keras.backend.mean(tensorflow.keras.backend.square(X_train-y_predict), axis=[1, 2, 3])
        return reconstruction_loss_factor * reconstruction_loss

    def vae_kl_loss(encoder_mu, encoder_log_variance):
        kl_loss = -0.5 * tensorflow.keras.backend.sum(1.0 + encoder_log_variance -
                                                      tensorflow.keras.backend.square(encoder_mu) -
                                                      tensorflow.keras.backend.exp(encoder_log_variance), axis=1)
        return kl_loss

    def vae_kl_loss_metric(X_train, y_predict):
        kl_loss = -0.5 * tensorflow.keras.backend.sum(1.0 + encoder_log_variance -
                                                      tensorflow.keras.backend.square(encoder_mu) -
                                                      tensorflow.keras.backend.exp(encoder_log_variance), axis=1)
        return kl_loss

    def vae_loss(X_train, y_predict):
        reconstruction_loss = vae_reconstruction_loss(X_train, y_predict)
        kl_loss = vae_kl_loss(X_train, y_predict)

        loss = reconstruction_loss + kl_loss
        return loss

    return vae_loss

In [ ]:
def calculating_metrics(vae,decoder,encoder):
  decoder.trainable = True
  encoder.trainable = True
  decoder_output = decoder(encoder.output)
  model = models.Model(encoder.input, decoder_output)
  model.compile(optimizer=tensorflow.keras.optimizers.Adam(lr=0.0005), loss=loss_function(encoder_mu, encoder_log_variance))
  return model


>GAN

In [ ]:
def define_gan(vae1, d_model, encoder, decoder):

	d_model.trainable = False
	gan_output = d_model(vae1.output) 
	model = models.Model(vae1.input, gan_output)
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

>load real samples

In [ ]:
def load_real_samples():

	(trainX, trainy), (_, _) = load_data()
	X = np.expand_dims(trainX, axis=-1)
	X = X.astype('float32')
	X = (X - 127.5) / 127.5
	print(X.shape, trainy.shape)
	return [X, trainy]

>select supervised samples

In [ ]:
def select_supervised_samples(dataset, n_samples=100, n_classes=10):
	X, y = dataset
	X_list, y_list = list(), list()
	n_per_class = int(n_samples / n_classes)
	for i in range(n_classes):
		X_with_class = X[y == i]
		ix = randint(0, len(X_with_class), n_per_class)
		[X_list.append(X_with_class[j]) for j in ix]
		[y_list.append(i) for j in ix]
	return np.asarray(X_list), np.asarray(y_list)

> generate_real_sample

In [ ]:
def generate_real_samples(dataset, n_samples):
	images, labels = dataset
	ix = randint(0, images.shape[0], n_samples)
	X, labels = images[ix], labels[ix]
	y = np.ones((n_samples, 1))
	return [X, labels], y

>generate_latent_points

In [ ]:
def generate_latent_points(latent_dim, n_samples):
	z_input = randn(latent_dim * n_samples)
	z_input = z_input.reshape(n_samples, latent_dim)
	return z_input

>generate_fake_samples

In [ ]:
def generate_fake_samples(VAE, encoder, decoder, dataset, n_samples=100):
	
  X, y = dataset
  X_list, y_list = np.zeros((n_samples,28,28,1)), list()
  
  number = np.random.choice(np.arange(len(X)),round(n_samples),replace=False)
  for i in range(n_samples):
    X_list[i,:,:,:] = X[number[i],:,:,:]
  fake_images = VAE.predict(X_list.reshape(n_samples,28,28,1))
	
  y = np.zeros((n_samples, 1)) 
  return fake_images, y

>summarize_performance

In [ ]:
def summarize_performance(step, vae, encoder, decoder, c_model,  dataset, n_samples=100):
  X_unsup, y_unsup = select_supervised_samples(dataset)
  X, y = generate_fake_samples(vae,encoder,decoder,dataset,n_samples)
	
  X = (X + 1) / 2.0 
	
  for i in range(50):
    plt.subplot(10, 10, 1 + i)
    plt.axis('off')
    plt.imshow(X[i, :, :, 0], cmap='gray_r')
  
  filename1 = 'generated_plot_%04d.png' % (step+1)
  plt.savefig(filename1)
  plt.close()
    
  
  X, y = dataset
  _, acc = c_model.evaluate(X, y, verbose=0)
  print('Discriminator Accuracy: %.3f%%' % (acc * 100))
  
  filename2 = 'gen_model_%04d.h5' % (step+1)
  vae.save(filename2)
  
  filename3 = 'disc_sup_%04d.h5' % (step+1)
  c_model.save(filename3)
  print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

>Training

In [ ]:
def train(vae, encoder, decoder, d_model, c_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=100):
		X_sup, y_sup = select_supervised_samples(dataset)
		bat_per_epo = int(dataset[0].shape[0] / n_batch)
		n_steps = bat_per_epo * n_epochs
		half_batch = int(n_batch / 2)
		for i in range(n_steps):
			[Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
			sup_loss, sup_acc = c_model.train_on_batch(Xsup_real, ysup_real)
			[X_real, _], y_real = generate_real_samples(dataset, half_batch)
			d_loss_real = d_model.train_on_batch(X_real, y_real)
			X_fake, y_fake = generate_fake_samples(vae1, encoder,decoder,dataset, half_batch)
			d_loss_fake = d_model.train_on_batch(X_fake, y_fake)
			X_gan, y_gan = X_real , np.ones((half_batch, 1))
			gan_loss = gan_model.train_on_batch(X_gan, y_gan)
			vae_loss = metr_model.train_on_batch(X_gan,X_gan)
			print('>%d, c[%.3f,%.0f], disc_loss[%.3f,%.3f], gan_loss[%.3f], vae_loss[%.3f]' % 
						(i+1, sup_loss, sup_acc*100, d_loss_real, d_loss_fake, gan_loss, vae_loss))
			if (i+1) % (bat_per_epo * 1) == 0:
				summarize_performance(i, vae, encoder,decoder,c_model,dataset, half_batch)

> Run

In [ ]:
latent_dim = 100
unsup_model, sup_model = define_discriminator() 
gen_model = vae1 
gan_model = define_gan(gen_model, unsup_model ,encoder,decoder) 
metr_model = calculating_metrics(vae,decoder,encoder)
dataset = load_real_samples()
train(gen_model,encoder,decoder, unsup_model, sup_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=100)

Streaming output truncated to the last 5000 lines.
>7028, c[0.000,100], disc_loss[0.000,0.035], gan_loss[14.352], vae_loss[199.382]
>7029, c[0.000,100], disc_loss[0.000,0.000], gan_loss[14.750], vae_loss[224.589]
>7030, c[0.000,100], disc_loss[0.012,0.000], gan_loss[13.657], vae_loss[228.536]
>7031, c[0.001,100], disc_loss[0.000,0.011], gan_loss[15.040], vae_loss[217.040]
>7032, c[0.002,100], disc_loss[0.001,0.000], gan_loss[14.684], vae_loss[216.915]
>7033, c[0.000,100], disc_loss[0.004,0.000], gan_loss[14.524], vae_loss[200.285]
>7034, c[0.001,100], disc_loss[0.041,0.000], gan_loss[13.902], vae_loss[209.912]
>7035, c[0.000,100], disc_loss[0.000,0.000], gan_loss[14.220], vae_loss[215.291]
>7036, c[0.000,100], disc_loss[0.000,0.000], gan_loss[13.022], vae_loss[215.603]
>7037, c[0.000,100], disc_loss[0.000,0.001], gan_loss[12.505], vae_loss[213.540]
>7038, c[0.000,100], disc_loss[0.000,0.003], gan_loss[13.381], vae_loss[219.309]
>7039, c[0.000,100], disc_loss[0.000,0.002], gan_loss[12.8

>evaluate the model

In [85]:
from keras.models import load_model

disc_sup_trained_model = load_model('disc_sup_12000.h5')
(trainX, trainy), (testX, testy) = load_data()
testX = np.expand_dims(testX, axis=-1)
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
trainX = trainX.astype('float32')
trainX = (trainX - 127.5) / 127.5
_, test_acc = disc_sup_trained_model.evaluate(testX, testy, verbose=0)
print('Test Accuracy: %.3f%%' % (test_acc * 100))
_, train_acc =  disc_sup_trained_model.evaluate(trainX, trainy, verbose=0)
print('Train Accuracy: %.3f%%' % (train_acc * 100))

Test Accuracy: 77.590%
Train Accuracy: 76.005%
